# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: 
* Model: 
* Evaluation approach: 
* Fine-tuning dataset: 

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, Trainer, TrainingArguments
import numpy as np

In [ ]:
#import sms_spam data set
# https://huggingface.co/datasets/mteb/tweet_sentiment_extraction
splits = ["train", "test"]

dataset = {split: load_dataset("mteb/tweet_sentiment_extraction", split=split) for split in splits}

#check dataset
print("Dataset: ", dataset["train"])

#inspect dataset
print("Dataset sample:", dataset["train"][0])

In [ ]:
#define model name
model_name = "distilbert-base-uncased"
#load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

#define preprocess function
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

tokenized_ds = {}
for split in splits:
    tokenized_ds[split] = dataset[split].map(preprocess_function, batched=True)

#check tockenized ds
print("Tokenized DS: ", tokenized_ds["train"])

# Show the first example of the tokenized training set
print("Tokenized Input ids: ", tokenized_ds["train"][0]["input_ids"])

In [ ]:
num_labels=3
id2label={0: 'negative', 1: 'neutral', 2: 'positive'}
label2id={'negative': 0, 'neutral': 1, 'positive':2}

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)
print("Model: ", model)

# Freeze the base model parameters
for param in model.base_model.parameters():
    param.requires_grad = False

In [ ]:
# train model

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

training_args = TrainingArguments(
    output_dir="./ouput/"+model_name,
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    learning_rate=2e-5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=1,
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
prefine_tuning_results = trainer.evaluate()
print("Evaluation results before fine-tuning: ", prefine_tuning_results)



## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.